In [1]:
import tensorflow as tf
import pandas as pd


train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

y_train = train['label']
x_train = train.drop(['label'], axis=1)

# y_test = test['label']
x_test = test

In [2]:
import numpy as np

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = np.array(x_train).reshape(x_train.shape[0], 28, 28, 1)
x_test = np.array(x_test).reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (42000, 28, 28, 1)
Number of images in x_train 42000
Number of images in x_test 28000


In [3]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

# Creating a Sequential Model and adding the layers
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), padding='same', activation='linear', input_shape=input_shape))
model.add(BatchNormalization(axis=-1))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(56, kernel_size=(3,3), padding='same', activation='relu',))
model.add(Conv2D(112, kernel_size=(3,3), padding='same', activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(112, kernel_size=(3,3), padding='same', activation='relu',))
model.add(Conv2D(112, kernel_size=(3,3), padding='same', activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))

Using TensorFlow backend.


In [4]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=30)


Epoch 1/30
42000/42000 [==============================] - 372s 9ms/step - loss: 0.1740 - accuracy: 0.9460
Epoch 2/30
42000/42000 [==============================] - 372s 9ms/step - loss: 0.0587 - accuracy: 0.9833
Epoch 3/30
42000/42000 [==============================] - 370s 9ms/step - loss: 0.0443 - accuracy: 0.9869
Epoch 4/30
42000/42000 [==============================] - 370s 9ms/step - loss: 0.0363 - accuracy: 0.9897
Epoch 5/30
42000/42000 [==============================] - 373s 9ms/step - loss: 0.0296 - accuracy: 0.9917
Epoch 6/30
42000/42000 [==============================] - 370s 9ms/step - loss: 0.0271 - accuracy: 0.9919
Epoch 7/30
42000/42000 [==============================] - 369s 9ms/step - loss: 0.0233 - accuracy: 0.9925
Epoch 8/30
42000/42000 [==============================] - 371s 9ms/step - loss: 0.0205 - accuracy: 0.9937
Epoch 9/30
42000/42000 [==============================] - 370s 9ms/step - loss: 0.0188 - accuracy: 0.9943
Epoch 10/30
42000/42000 [=====================

In [5]:
# predict results
results = model.predict(x_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_V3.csv",index=False)